# Usage examples of `swissparlpy`

In [ ]:
import swissparlpy
import requests
import pandas as pd
import os
import urllib3
from datetime import datetime

In [ ]:
urllib3.disable_warnings()
__location__ = os.path.realpath(os.getcwd())

## Create client with custom session

Sometimes it's necessary to tweak the requests Session (e.g. to provide authentication or disable SSL verification).
For this purpose a custom session can be passed to `SwissParlClient`.

In [ ]:
session = requests.Session()
session.verify = False # disable SSL verification
client = swissparlpy.SwissParlClient(session=session)

For most common cases, this is not necessary and you don't even have to create your own `SwissParlClient`.

Simply use the shorthand methods to get the data:

In [ ]:
import swissparlpy as spp

tables = spp.get_tables()
glimpse_df = pd.DataFrame(spp.get_glimpse(tables[0]))
glimpse_df

## Get metadata of tables and variables

In [ ]:
client.get_tables() # get list of all tables

In [ ]:
client.get_variables('Party') # get list of variables of a table

## Load data in `pandas` DataFrame

In [ ]:
parties = client.get_data('Party', Language='DE')
parties_df = pd.DataFrame(parties)
parties_df

## Use substring operators to query data (`__startswith`, `__contains`)

In [ ]:
persons = client.get_data("Person", Language="DE", LastName__startswith='Bal')
persons.count

In [ ]:
person_df = pd.DataFrame(persons)
person_df

In [ ]:
co2_business = client.get_data("Business", Title__contains="CO2", Language = "DE")
co2_business.count

In [ ]:
co2_df = pd.DataFrame(co2_business)
co2_df

## Query with date ranges

In [ ]:
business_oct19 = client.get_data(
    "Business",
    Language="DE",
    SubmissionDate__gte=datetime.fromisoformat('2019-10-01'),
    SubmissionDate__lt=datetime.fromisoformat('2019-10-31')
)
business_oct19.count

In [ ]:
busi_oct19 = pd.DataFrame(business_oct19)
busi_oct19 = busi_oct19.sort_values(by=['SubmissionDate']).reset_index(drop=True)
busi_oct19[['SubmissionDate', 'Title']]

## Download large query in batches

This script shows how to download votes from the `Voting` table by iterating over each session in a legislative period.
The chunks are then saved in a directory as pickled DataFrames.

Later on, those chunks can easily be combined together as a single DataFrame containing all the votes of a legislative period.

In [ ]:
path = os.path.join(__location__, "voting50")

def save_votes_of_session(id):
    if not os.path.exists(path):
        os.mkdir(path)
    pickle_path = os.path.join(path, f'{id}.pks')
    
    if os.path.exists(pickle_path):
        print(f"File {pickle_path} already exists, skipping")
        return
    
    print(f"Loading votes of session {id}...")
    data = client.get_data("Voting", Language="DE", IdSession=id)
    print(f"{data.count} rows loaded.")
    df = pd.DataFrame(data)
    
    df.to_pickle(pickle_path)
    print(f"Saved pickle at {pickle_path}")
    print("")


# get all session of the 50 legislative period
sessions50 = client.get_data("Session", Language="DE", LegislativePeriodNumber=50)
sessions50.count

for session in sessions50:
    print(f"Loading session {session['ID']}")
    save_votes_of_session(session['ID'])

# Combine to one dataframe
path = os.path.join(__location__, "voting50")
df_voting50 = pd.concat([pd.read_pickle(os.path.join(path, x)) for x in os.listdir(path)])
df_voting50

In [ ]:
df_5005 = pd.read_pickle(os.path.join(__location__, "voting50", '5005.pks'))
df_5005

In [ ]:
# Combine to one dataframe
path = os.path.join(__location__, "voting50")
df_voting50 = pd.concat([pd.read_pickle(os.path.join(path, x)) for x in os.listdir(path)])
df_voting50